# Vestibular Question Answuring Dataset - VQuAD

##### Inicialmente se extajo del vestibular una serie de contextos basados en los parrafos y secciones de texto que componen el documento. Los contextos se extrajeron com base a los titulos que pertenecian. se itentificaron 28 titulos para 473 contextos.

## Load contextos

In [1]:
import pandas as pd

# Read the dataset
df_context = pd.read_csv('contextos.csv')
df_context.columns = ['title', 'context']
df_context.head()

,title,context
0,titulo,Vestibular Unicamp 2024\nResolução GR-031/2023...
1,Vagas e sistemas de ingresso à Graduação,Art. 1º Para o ano de 2024 são oferecidas 3340...
2,Vagas e sistemas de ingresso à Graduação,§1º As vagas regulares não preenchidas nos inc...
3,Vagas e sistemas de ingresso à Graduação,"§2º As 95 vagas adicionais, indicadas nos inci..."
4,Vagas e sistemas de ingresso à Graduação,§3º Na hipótese de cancelamento dos sistemas d...


Context count

In [2]:
ncontextos = len(df_context)
ntitulos = len(df_context['title'].unique())

print(f"""
      **Esta base de dados contem \033[92m{ncontextos}\033[0m contextos de \033[92m{ntitulos}\033[0m titulos do vestibular da Unicamp para o ano 2024.**
      """)


      **Esta base de dados contem 471 contextos de 28 titulos do vestibular da Unicamp para o ano 2024.**
      


Titulos

In [3]:
titulos = df_context["title"].unique()
for i in titulos:
    print(f"\033[1m{i}\033[0m")

titulo
Vagas e sistemas de ingresso à Graduação 
Objetivo e características do Vestibular Unicamp
Sobre vagas oferecidas e as modalidades de classiﬁcação 
Inscrição
Sobre as provas, notas e convocações
Matrículas
Sobre as prioridades nos sistemas de ingresso Unicamp
Disposições gerais
ANEXO I
ANEXO II - PROGRAMA DAS PROVAS
ANEXO II - REDAÇÃO, LÍNGUA PORTUGUESA E LITERATURAS DE LÍNGUA PORTUGUESA
ANEXO II - RELAÇÃO DE LIVROS
ANEXO II - MATEMÁTICA
ANEXO II - GEOGRAFIA
ANEXO II - HISTÓRIA
ANEXO II - SOCIOLOGIA
ANEXO II - FILOSOFIA
ANEXO II - LÍNGUA INGLESA
ANEXO II - CIÊNCIAS BIOLÓGICAS
ANEXO II - FÍSICA
ANEXO II - QUÍMICA
ANEXO II - PROVAS DE HABILIDADES ESPECÍFICAS
ANEXO III
ANEXO IV - Indicação de vagas e critérios para remanejamento de vagas no curso de Música
ANEXO V - Cursos de graduação por área de realização da prova do Vestibular Unicamp.
ANEXO VI - Candidatos(as) com deficiência ou em condições que exijam recursos específicos para realizar as provas do VU 2024
ANEXO VII - Redução

## Estabelecemos o prompt do sistema

```python
example_conversation = {
        "systema":
        """
        Você é um anotador de dados.
        Dado um contexto o anotador deve fornecer três perguntas e três respostas para cada pergunta.
        As perguntas devem esclareçam possíveis dúvidas de um usuário sobre esse contexto.
        As respostas devem solucionar a pergunta usando o mesmo contexto.
        As respostas devem estar contida no contexto.
        As respostas devem ser sucintas.
        As respostas devem ser três para a mesma pergunta, onde:
            a primeira deve ser simples y dar solução na pergunta sim muito argumento,
            a segunda deve ter um argumento melhor e
            a terceira resposta deve ser mais completa.
        
        exemplo:
            contexto: Em muitas instituições, o TCC é encarado como critério final de avaliação do aluno: em caso de reprovação, o aluno estará impedido de obter o diploma e consequentemente exercer a respectiva profissão até que seja aprovado. O escopo e o formato do TCC (assim como sua própria nomenclatura) variam entre os diversos cursos e entre diferentes instituições, mas na estrutura curricular brasileira ela possui papel de destaque: em cursos ligados às ciências, normalmente é um trabalho que envolve pesquisa experimental, em cursos de caráter profissional, normalmente envolve: pesquisa bibliográfica e/ou empírica, a execução em si e uma apresentação de um projeto perante uma banca examinadora entre 3 e 5 professores (não necessariamente com Mestre ou Doutor). A banca examinadora formada para tal propósito não cria nenhuma expectativa de originalidade. Portanto, pode ser uma compilação (e não cópia) de outros ensaios com uma finalidade, um fio condutor, ou algo que forneça um roteiro, uma continuidade.
            anotação: 
            ```
                {
                    perguntas: {
                        0: "O que acontece se o aluno reprovar no TCC?",
                        1: "Como o TCC é compreendida em instituições?",
                        2: "Quantos professores compõem a banca examinadora?"
                    }
                    respostas: {
                        0: ["o aluno estará impedido de obter o diploma",
                            "estará impedido de obter o diploma e consequentemente exercer a respectiva profissão",
                            "estará impedido de obter o diploma" ],
                        1: ["como critério final de avaliação do aluno",
                            "como critério final de avaliação do aluno",
                            "como critério final de avaliação do aluno" ],
                        2: ["entre 3 e 5",
                            "entre 3 e 5 professores",
                            "entre 3 e 5 professores (não necessariamente com Mestre ou Doutor)"],
                }
            ```
        """
    }
```

In [4]:
system = {
    "role": "system",
    "content":  """
                Você é um anotador de dados.
                Dado um contexto o anotador deve fornecer três perguntas e três respostas para cada pergunta.
                As perguntas devem esclareçam possíveis dúvidas de um usuário sobre esse contexto.
                As respostas devem solucionar a pergunta usando o mesmo contexto.
                As respostas devem estar contida no contexto.
                As respostas devem ser sucintas.
                As respostas devem ser três para a mesma pergunta, onde:
                    a primeira deve ser simples y dar solução na pergunta sim muito argumento,
                    a segunda deve ter um argumento melhor e
                    a terceira resposta deve ser mais completa.

                exemplo:
                    contexto: Em muitas instituições, o TCC é encarado como critério final de avaliação do aluno: em caso de reprovação, o aluno estará impedido de obter o diploma e consequentemente exercer a respectiva profissão até que seja aprovado. O escopo e o formato do TCC (assim como sua própria nomenclatura) variam entre os diversos cursos e entre diferentes instituições, mas na estrutura curricular brasileira ela possui papel de destaque: em cursos ligados às ciências, normalmente é um trabalho que envolve pesquisa experimental, em cursos de caráter profissional, normalmente envolve: pesquisa bibliográfica e/ou empírica, a execução em si e uma apresentação de um projeto perante uma banca examinadora entre 3 e 5 professores (não necessariamente com Mestre ou Doutor). A banca examinadora formada para tal propósito não cria nenhuma expectativa de originalidade. Portanto, pode ser uma compilação (e não cópia) de outros ensaios com uma finalidade, um fio condutor, ou algo que forneça um roteiro, uma continuidade.
                    anotaição: 
                    ```
                        {
                            "perguntas": {
                                "p0": "O que acontece se o aluno reprovar no TCC?",
                                "p1": "Como o TCC é compreendida em instituições?",
                                "p2": "Quantos professores compõem a banca examinadora?"
                            }
                            "respostas": {
                                "p0": {
                                    "r0" : "o aluno estará impedido de obter o diploma",
                                    "r1" : "estará impedido de obter o diploma e consequentemente exercer a respectiva profissão",
                                    "r2" : "estará impedido de obter o diploma",
                                    }
                                "p1": {
                                    "r0" : "como critério final de avaliação do aluno",
                                    "r1" : "como critério final de avaliação do aluno",
                                    "r2" : "como critério final de avaliação do aluno",
                                    }
                                    
                                "p2": {
                                    "r0" : "entre 3 e 5",
                                    "r1" : "entre 3 e 5 professores",
                                    "r2" : "entre 3 e 5 professores (não necessariamente com Mestre ou Doutor)",
                                    }
                            }
                        }
                    ```
                """
    }

In [5]:
# import openai
from openai import OpenAI
import os

def join(context: dict, content: dict, data: list):
    
    for i in content["perguntas"]:
        # print(i)
        # print(list(content["respostas"][i].values()))
        d = {
            "title": context["title"],
            "context": context["context"],
            "pergunta": content["perguntas"][i],
            "resposta": content["respostas"][i]
        }
        data.append(d)
    return data

def obtener_respuesta_gpt(user_input):
    # Llama a la API de GPT-3.5 Turbo para obtener una respuesta
    # Utiliza la biblioteca openai y tu clave de API
    
    client = OpenAI(
        api_key= os.getenv("OPENAI_API_KEY"),
    )
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=user_input,
    )
    return response.choices[0].message.content

In [8]:
import json

def dataframe_request(df: pd.DataFrame, request: list, m_max: int = 7):
    data = df.to_dict(orient="records")
    df_data = []
    
    for d in data:
        if len(request) == m_max:
            request.pop(1)
            request.pop(2)
        
        request.append({
            "role": "user",
            "content": f"Contexto: {d['context']}\nanotaição:"
        })
        
        # print(request)
        response = obtener_respuesta_gpt(request)
        chatbot_message = {"role": "assistant", "content": response}
        request.append(chatbot_message)
        df_data = join(
            context=d,
            content=eval(response),
            data=df_data)
    
    return pd.DataFrame(df_data)

m_max = 5
request = [system]
df_c = df_context.copy()

df = dataframe_request(df_c, request, m_max)
df


{
    "perguntas": {
        "p0": "Qual é a data de publicação da Resolução?",
        "p1": "Quem é o reitor em exercício?",
        "p2": "O que a Resolução especifica?"
    },
    "respostas": {
        "p0": {
            "r0": "13/07/2023",
            "r1": "13/07/2023",
            "r2": "13/07/2023"
        },
        "p1": {
            "r0": "Maria Luiza Moretti",
            "r1": "Maria Luiza Moretti",
            "r2": "Maria Luiza Moretti"
        },
        "p2": {
            "r0": "sobre os sistemas de ingresso aos Cursos de Graduação da Unicamp",
            "r1": "sobre os sistemas de ingresso aos Cursos de Graduação da Unicamp",
            "r2": "sobre os sistemas de ingresso aos Cursos de Graduação da Unicamp"
        }
    }
}
{
    "perguntas": {
        "p0": "Quantas vagas regulares são oferecidas para ingresso nos Cursos de Graduação da Unicamp em 2024?",
        "p1": "Quais são os sistemas de ingresso para as vagas regulares?",
        "p2": "Quantas vagas

,title,context,pergunta,resposta
0,titulo,Vestibular Unicamp 2024\nResolução GR-031/2023...,Qual é a data de publicação da Resolução?,"{'r0': '13/07/2023', 'r1': '13/07/2023', 'r2':..."
1,titulo,Vestibular Unicamp 2024\nResolução GR-031/2023...,Quem é o reitor em exercício?,"{'r0': 'Maria Luiza Moretti', 'r1': 'Maria Lui..."
2,titulo,Vestibular Unicamp 2024\nResolução GR-031/2023...,O que a Resolução especifica?,{'r0': 'sobre os sistemas de ingresso aos Curs...
3,Vagas e sistemas de ingresso à Graduação,Art. 1º Para o ano de 2024 são oferecidas 3340...,Quantas vagas regulares são oferecidas para in...,"{'r0': '3340 vagas', 'r1': '3340 vagas', 'r2':..."
4,Vagas e sistemas de ingresso à Graduação,Art. 1º Para o ano de 2024 são oferecidas 3340...,Quais são os sistemas de ingresso para as vaga...,"{'r0': 'Vestibular Unicamp (VU) 2024, Edital E..."
5,Vagas e sistemas de ingresso à Graduação,Art. 1º Para o ano de 2024 são oferecidas 3340...,Quantas vagas são oferecidas pelo Vestibular U...,"{'r0': '2537 vagas', 'r1': '2537 vagas', 'r2':..."
6,Vagas e sistemas de ingresso à Graduação,§1º As vagas regulares não preenchidas nos inc...,O que acontece com as vagas regulares não pree...,{'r0': 'As vagas são transferidas para o VU 20...
7,Vagas e sistemas de ingresso à Graduação,§1º As vagas regulares não preenchidas nos inc...,Quantas vagas são oferecidas no total para ing...,"{'r0': '3340 vagas', 'r1': '3340 vagas', 'r2':..."
8,Vagas e sistemas de ingresso à Graduação,§1º As vagas regulares não preenchidas nos inc...,Qual é o sistema de ingresso prioritário para ...,"{'r0': 'Vestibular Unicamp (VU) 2024', 'r1': '..."
9,Vagas e sistemas de ingresso à Graduação,"§2º As 95 vagas adicionais, indicadas nos inci...",O que acontece com as 95 vagas adicionais indi...,{'r0': 'As vagas não serão transferidas para o...


In [ ]:
a = df_context.iloc[5].to_dict()
d = eval(messages[2]["content"])
# datata = []

datata = join(a, d, datata)
    
dff = pd.DataFrame(datata)
dff

,title,context,pergunta,resposta
0,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,O que acontece com as vagas não preenchidas no...,{'r0': 'As vagas não preenchidas nos sistemas ...
1,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,Quantas vagas são oferecidas para o ingresso n...,{'r0': 'São oferecidas 3340 vagas para o ingre...
2,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,Qual é o sistema que recebe as vagas não preen...,{'r0': 'O sistema de ingresso que recebe as va...
3,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,O que acontece com as vagas não preenchidas no...,{'r0': 'As vagas não preenchidas nos sistemas ...
4,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,Quantas vagas são oferecidas para o ingresso n...,{'r0': 'São oferecidas 3340 vagas para o ingre...
5,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,Qual é o sistema que recebe as vagas não preen...,{'r0': 'O sistema de ingresso que recebe as va...
6,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,O que acontece com as vagas não preenchidas no...,{'r0': 'As vagas não preenchidas nos sistemas ...
7,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,Quantas vagas são oferecidas para o ingresso n...,{'r0': 'São oferecidas 3340 vagas para o ingre...
8,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,Qual é o sistema que recebe as vagas não preen...,{'r0': 'O sistema de ingresso que recebe as va...
9,Vagas e sistemas de ingresso à Graduação,§4º No caso de transferência de vagas para o V...,O que acontece com as vagas não preenchidas no...,{'r0': 'As vagas não preenchidas nos sistemas ...


In [59]:
df = pd.DataFrame()
for i in messages:
    if i["role"] == "assistant":
        print("assistant")
        print(type(i["content"]))
        d = eval(i["content"])
        print(json.dumps(d, indent=4))
        print()
        df = pd.concat([df, pd.DataFrame(d)], axis=0)
        
        
    # if i["role"] == "user":
    #     print("user")
    #     print(i["content"])
    #     print()
df

assistant
<class 'str'>
{
    "perguntas": {
        "p0": "O que acontece com as vagas n\u00e3o preenchidas nos sistemas de ingresso II, III, IV e V?",
        "p1": "Quantas vagas s\u00e3o oferecidas para o ingresso na gradua\u00e7\u00e3o?",
        "p2": "Qual \u00e9 o sistema que recebe as vagas n\u00e3o preenchidas nos outros sistemas de ingresso?"
    },
    "respostas": {
        "p0": {
            "r0": "As vagas n\u00e3o preenchidas nos sistemas de ingresso II, III, IV e V s\u00e3o transferidas para o VU 2024",
            "r1": "As vagas n\u00e3o preenchidas nos sistemas de ingresso II, III, IV e V s\u00e3o disponibilizadas para o VU 2024",
            "r2": "As vagas n\u00e3o preenchidas nos sistemas de ingresso II, III, IV e V s\u00e3o destinadas ao VU 2024"
        },
        "p1": {
            "r0": "S\u00e3o oferecidas 3340 vagas para o ingresso na gradua\u00e7\u00e3o",
            "r1": "A gradua\u00e7\u00e3o oferece um total de 3340 vagas",
            "r2": "O ingre

,perguntas,respostas
p0,O que acontece com as vagas não preenchidas no...,{'r0': 'As vagas não preenchidas nos sistemas ...
p1,Quantas vagas são oferecidas para o ingresso n...,{'r0': 'São oferecidas 3340 vagas para o ingre...
p2,Qual é o sistema que recebe as vagas não preen...,{'r0': 'O sistema de ingresso que recebe as va...
p0,O que acontece com as 95 vagas adicionais indi...,{'r0': 'As 95 vagas adicionais não serão trans...
p1,As vagas adicionais indicadas nos incisos IV e...,{'r0': 'As vagas adicionais indicadas nos inci...
p2,Essas vagas adicionais são de preenchimento ob...,{'r0': 'As vagas adicionais indicadas nos inci...
p0,O que acontece se os sistemas de ingresso prev...,{'r0': 'Se os sistemas de ingresso forem cance...
p1,Onde serão transferidas as vagas regulares em ...,{'r0': 'As vagas regulares serão transferidas ...
p2,"Em caso de cancelamento, haverá reserva de vag...","{'r0': 'Em caso de cancelamento, haverá reserv..."


In [9]:
df_context.to_dict(orient="records")

[{'title': 'titulo',
  'context': 'Vestibular Unicamp 2024\nResolução GR-031/2023, de 13/07/2023\nReitor: Maria Luiza Moretti - Reitora em exercício\nO Reitor da Universidade Estadual de Campinas, considerando a Deliberação CONSU-A-032/2017 de 21 de novembro de 2017, que especiﬁca sobre os sistemas de ingresso aos Cursos de Graduação da Unicamp, torna pública a Resolução Vestibular Unicamp 2024 para vagas no ensino de Graduação.'},
 {'title': 'Vagas e sistemas de ingresso à Graduação\xa0',
  'context': 'Art. 1º Para o ano de 2024 são oferecidas 3340 vagas regulares para ingresso nos Cursos de Graduação da Unicamp distribuídas nos seguintes sistemas de ingresso:\xa0 I. 2537 vagas oferecidas pelo Vestibular Unicamp (VU) 2024.\xa0 II. 314 vagas oferecidas pelo Edital ENEM-Unicamp 2024.\xa0 III. 325 vagas oferecidas pelo Provão Paulista 2024.\xa0 IV. 49 vagas oferecidas pelo Vestibular Indígena (VI) 2024. O Vestibular Indígena terá ainda 81 vagas adicionais, conforme Edital a ser publicado

In [1]:
import pandas as pd
pd.DataFrame([(0, "H"), (1, "O"), (2, "L"), (3, "A")], columns=["id", "name"])

,id,name
0,0,H
1,1,O
2,2,L
3,3,A
